**IMPORTS**

In [27]:
import os
import math
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from scipy.signal import hilbert, butter, lfilter
import soundfile as sf

**RUTAS Y TIPOS DE FRUTAS**

In [8]:
fruit_types      = ['pera', 'banana', 'manzana', 'naranja']
audios           = {fruit: [] for fruit in fruit_types}
dataset_path     = '../../dataset'
original_path    = os.path.join(dataset_path, 'original')
processed_path   = os.path.join(dataset_path, 'processed')

**DICCIONARIO DE AUDIOS ORIGINALES**

In [36]:
original = {fruit: [] for fruit in fruit_types}
for dirname, _, filenames in os.walk(original_path):
    subdir = os.path.basename(dirname)
    if subdir in fruit_types:
        original[subdir].extend([os.path.join(dirname, filename) for filename in filenames if filename.endswith('.wav')])

**DICCIONARIO DE AUDIOS PROCESADOS**

In [35]:
processed = {fruit: [] for fruit in fruit_types}
for dirname, _, filenames in os.walk(processed_path):
    subdir = os.path.basename(dirname)
    if subdir in fruit_types:
        processed[subdir].extend([os.path.join(dirname, filename) for filename in filenames if filename.endswith('.wav')])

**PARAMETROS DEL AUDIO**

In [67]:
FRAME_SIZE = 1024# In the documentation says it's convenient for speech.C
HOP_SIZE   = int(FRAME_SIZE/2)

**FUNCIONES GENERALES DE AUDIO**

In [37]:
def load_audio(audiofile):
    test_audio, sr = librosa.load(audiofile, sr = None)
    duration = librosa.get_duration(filename=audiofile, sr=sr)
    return test_audio, sr, duration

In [38]:
def time_vector(signal, duration):
    return np.linspace(0, duration, len(signal))

In [65]:
def rms(signal, frames, hop):
    return librosa.feature.rms(y=signal, frame_length = frames, hop_length = hop)

In [63]:
def normalize(signal):
    peak = np.max(signal)
    signal/=peak
    return signal

In [41]:
def derivative(signal, duration):
    signal = signal.reshape(-1,)
    dy = np.gradient(signal, np.linspace(0, duration, len(signal)))
    return dy

**FILTERS**

In [42]:
def low_pass_filter(signal, sr, cutoff_frequency = 5000):
    nyquist = 0.5 * sr
    cutoff = cutoff_frequency / nyquist
    b, a = butter(N=6, Wn=cutoff, btype='low', analog=False, output='ba')
    filtered = lfilter(b, a, signal)
    return filtered

In [43]:
def band_pass_filter(signal, sr, low_cutoff, high_cutoff):
    b, a = butter(N=3, Wn = [low_cutoff, high_cutoff], btype='band', fs=sr)
    return lfilter(b, a, signal)

In [44]:
def preemphasis(signal, coef=0.97):
    return np.append(signal[0], signal[1:] - coef * signal[:-1])

In [45]:
def envelope(signal):
    analytic_signal = hilbert(signal)
    return np.abs(analytic_signal)

In [46]:
def smooth_envelope(signal, sr, cutoff_frequency=50.0):
    return low_pass_filter(envelope(signal), sr, cutoff_frequency)

**PRROCCESSING OF THE AUDIO FILES FUNCTIONS**

In [66]:
def process(audio_in, audio_out, umbral = 0.295):
    signal, sr, duration = load_audio(audio_in)
    
    filtered = low_pass_filter(signal, sr, 1800)
    filtered = preemphasis(filtered, 0.999)

    rms_signal = rms(signal, 4096, 2048)

    rms_signal = normalize(rms_signal)
    drms = normalize(derivative(rms_signal, duration))

    audio_vector = time_vector(signal, duration)
    drms_vector = time_vector(drms, duration)

    left_index = np.argmax(np.abs(drms) > umbral)
    rigth_index = len(drms) - 1 - np.argmax(np.abs(np.flip(drms)) > umbral)

    left_time = drms_vector[left_index]
    rigth_time = drms_vector[rigth_index]

    mask_vector = audio_vector >= left_time

    audio_vector = audio_vector[mask_vector]
    trimed_signal = signal[mask_vector]

    mask_vector = audio_vector <= rigth_time

    audio_vector = audio_vector[mask_vector]
    trimed_signal = trimed_signal[mask_vector]

    sf.write(audio_out, trimed_signal, sr)

In [69]:
def process_audios(original:dict, processed:dict):
    already_processed = []
    for group in processed.values():
        already_processed.extend(group)
        
    for fruit, audios in original.items():
        for audio in audios:
            file = os.path.basename(audio)
            if file in already_processed:
                pass
            else:
                process(audio, os.path.join(processed_path, f"{fruit}/{file}"))

**PLOTTING**

In [30]:
#2d
def plot_features2d(features):
    fig = plt.figure()
    colors = dict(zip(fruit_types,['green','yellow','red','orange']))
    

    for fruit, points in features.items():
        plt.scatter(points[:, 0], points[:, 1], c = colors[fruit], label=fruit)

    plt.xlabel('Eje X')
    plt.ylabel('Eje Y')
    plt.show()

In [31]:
#3d
def plot_features3d(features):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    colors = dict(zip(fruit_types,['green','yellow','red','orange']))

    for fruit, points in features.items():
        ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=colors[fruit], marker='o', label=fruit)
        
    ax.set_xlabel('Eje X')
    ax.set_ylabel('Eje Y')
    ax.set_zlabel('Eje Z')
    plt.show()

**AUDIO PROCESSING**

In [70]:
process_audios(original, processed)

C:\Users\Juan\AppData\Local\Temp\ipykernel_5372\1587896153.py:3: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=audiofile, sr=sr)


**FEATURES EXTRACTION**

In [22]:
# Features extraction
features = dict.fromkeys(fruit_types)
split_frequency = 3000
cuton = 20
cutoff = 8500
n_mfcc = 4

for fruit, audios in processed.items():
    features[fruit] = None
    
    for audio in audios:
        # Load the audio signal
        signal, sr, duration = load_audio(audio)

        # Empty row of features
        feature = np.empty((1, 0))

        # Calculate the rms
        audio_rms = np.sqrt(np.mean(signal**2))/np.max(signal)
        feat = audio_rms
        feature = np.append(feature, audio_rms)

        # BER min
        spec = librosa.stft(signal, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
        BER  = band_energy_ratio(spec, split_frequency, sr)
        feat = np.min(BER)
        #feature = np.append(feature, feat)

        # Centroid
        centroid = librosa.feature.spectral_centroid(y=signal, sr=sr, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)[0]
        centroid /= np.max(np.abs(centroid))
        # Varnz
        feat = np.var(centroid)
        #feature = np.append(feature, feat)
        # std
        feat = np.std(centroid)/np.mean(centroid)
        feature = np.append(feature, feat)

        # Envelope RMS
        smoothed = rms(signal)
        smoothed = smoothed.reshape(-1,)
        smoothed /= np.max(np.abs(smoothed))
        # varnz
        feat = np.var(smoothed)
        #feature = np.append(feature, feat)
        #std
        feat = np.std(smoothed)/np.mean(smoothed)
        feature = np.append(feature, feat)
        #momentum
        t = time_vector(smoothed, duration)
        feat = np.dot(smoothed, t)/np.sum(smoothed)
        feature = np.append(feature, feat)

        #ZCR
        filtered = band_pass_filter(signal, sr, cuton, cutoff)
        zcr = librosa.feature.zero_crossing_rate(filtered, frame_length=FRAME_SIZE, hop_length=HOP_SIZE)[0]
        zcr /= np.max(np.abs(zcr))
        #mean
        feat = np.mean(zcr)
        feature = np.append(feature, feat)
        #maximum
        feat = np.max(zcr)
        #feature = np.append(feature, feat)
        #varnz
        feat = np.var(zcr)
        #feature = np.append(feature, feat)
        #std
        feat = np.std(zcr)/np.mean(zcr)
        feature = np.append(feature, feat)

        #MFCCS
        mfccs = librosa.feature.mfcc(y = signal, sr=sr, n_mfcc = n_mfcc, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
        #mean
        feat = np.mean(mfccs[:, ((mfccs.shape[1]*2)//5 - 5) : ((mfccs.shape[1]*2)//5 + 5)], axis = 1)
        feat = feat[1]
        feature = np.append(feature, feat)
        #maximum
        feat = np.max(mfccs[:, ((mfccs.shape[1]*2)//5 - 5) : ((mfccs.shape[1]*2)//5 + 5)], axis = 1)
        feat = feat[3]
        feature = np.append(feature, feat)
        
        mfccs /= np.max(np.abs(mfccs), axis = 1, keepdims=True)

        #vrnz
        feat = np.var(mfccs, axis = 1)
        feat = feat[1]
        #feature = np.append(feature, feat)
        #std
        feat = np.std(mfccs, axis = 1)/np.mean(mfccs, axis = 1)
        feat = feat[1]
        feature = np.append(feature, feat)
        #momentum
        frames = range(mfccs.shape[1])
        t = librosa.frames_to_time(frames, sr=sr, n_fft = FRAME_SIZE, hop_length = HOP_SIZE)
        feat = np.dot(mfccs, t)/np.sum(mfccs, axis = 1)
        feat = feat[0]
        feature = np.append(feature, feat)

        #hilbert envelope
        env = smooth_envelope(signal, sr, 45)
        selected = np.linspace(0, len(env) - 1, 30, dtype=int)
        env = env[selected]
        env = env.reshape(-1,1)
        feat = env[11]
        feature = np.append(feature, feat)
        feat = env[12]
        feature = np.append(feature, feat)

        if features[fruit] is not None:
            features[fruit] = np.vstack([features[fruit], feature])
        else:
            features[fruit] = feature

C:\Users\Juan\AppData\Local\Temp\ipykernel_4804\1587896153.py:3: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=audiofile, sr=sr)


In [26]:
#PCA
whole = np.concatenate(list(features.values()), axis=0)

#Paso 2: Aplicar PCA para obtener dos componentes principales
pca = PCA(n_components = 3)
reduced_features = pca.fit_transform(whole)

#Paso 3: Crear un diccionario con las matrices reducidas
reduced = {}
start_idx = 0

for fruit, matrix in features.items():
    num_rows = matrix.shape[0]
    reduced[fruit] = reduced_features[start_idx:start_idx + num_rows, :]
    start_idx += num_rows

In [ ]:
plot_features3d(reduced)